# Minimal PDF Chatbot with Gemini API
This notebook lets you upload a PDF and chat with it using Google's Gemini API. You can ask questions, request summaries, and more. The API key is loaded securely from a `.env` file.

## 1. Import Libraries and Load API Key

This cell imports all required libraries and loads your Gemini API key from the `.env` file.

In [2]:
import PyPDF2
import requests
from io import BytesIO
from IPython.display import display, Markdown
import ipywidgets as widgets
import os
from dotenv import load_dotenv

# Load Gemini API key from .env file
load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found. Please set it in a .env file.")
GEMINI_API_URL = 'https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent'

## 2. Gemini API and PDF Extraction Helpers

This cell defines the function to call Gemini and a helper to extract text from PDFs.

In [3]:
def ask_gemini(prompt, context=None):
    """Send a prompt (and optional context) to Gemini API and return the response text."""
    headers = {
        'Content-Type': 'application/json',
        'X-goog-api-key': GEMINI_API_KEY
    }
    if context:
        full_prompt = f"Context: {context}\n\nUser: {prompt}"
    else:
        full_prompt = prompt
    data = {
        "contents": [{"parts": [{"text": full_prompt}]}]
    }
    response = requests.post(GEMINI_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['candidates'][0]['content']['parts'][0]['text']
    else:
        return f"Error: {response.status_code} - {response.text}"

def extract_pdf_text(pdf_bytes):
    """Extract all text from a PDF file (as bytes)."""
    reader = PyPDF2.PdfReader(BytesIO(pdf_bytes))
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

## 3. Upload PDF and Extract Text

Use the widget below to upload your PDF. The text will be extracted and stored for chat context.

In [7]:
upload_widget = widgets.FileUpload(accept='.pdf', multiple=False)
display(Markdown('**Upload a PDF to begin chatting:**'))
display(upload_widget)

pdf_text = ''

def on_upload_change(change):
    global pdf_text
    if upload_widget.value:
        file_info = upload_widget.value[0]
        pdf_bytes = file_info['content']
        pdf_text = extract_pdf_text(pdf_bytes)
        display(Markdown(f"**PDF loaded. Extracted {len(pdf_text)} characters.**"))
        display(Markdown('You can now ask questions or request a summary.'))

upload_widget.observe(on_upload_change, names='value')

**Upload a PDF to begin chatting:**

FileUpload(value=(), accept='.pdf', description='Upload')

**PDF loaded. Extracted 111411 characters.**

You can now ask questions or request a summary.

## 4. Chat with the PDF using Gemini

Type your questions or requests below. The chatbot will use the extracted PDF text as context for Gemini.

In [8]:
input_box = widgets.Textarea(placeholder='Ask a question about the PDF, or type "summarize"...', description='You:')
submit_btn = widgets.Button(description='Send', button_style='primary')
output_area = widgets.Output()

chat_history = []

def on_submit(_):
    user_query = input_box.value.strip()
    if not user_query:
        return
    if not pdf_text:
        with output_area:
            display(Markdown('**Please upload a PDF first.**'))
        return
    chat_history.append({'role': 'user', 'content': user_query})
    with output_area:
        display(Markdown(f'**You:** {user_query}'))
        display(Markdown('**Gemini is thinking...**'))
    # Call Gemini API
    response = ask_gemini(user_query, context=pdf_text)
    chat_history.append({'role': 'gemini', 'content': response})
    with output_area:
        display(Markdown(f'**Gemini:** {response}'))
    input_box.value = ''
    output_area.clear_output(wait=True)
    # Re-display chat history
    for turn in chat_history:
        if turn['role'] == 'user':
            display(Markdown(f'**You:** {turn["content"]}'))
        else:
            display(Markdown(f'**Gemini:** {turn["content"]}'))

submit_btn.on_click(on_submit)
display(input_box, submit_btn, output_area)

Textarea(value='', description='You:', placeholder='Ask a question about the PDF, or type "summarize"...')

Button(button_style='primary', description='Send', style=ButtonStyle())

Output()

**You:** summarize the paper

**Gemini:** This paper explores the potential and challenges of adopting Artificial Intelligence (AI) in humanitarian practice, emphasizing the crucial role of Human-Computer Interaction (HCI) principles. Through thematic analysis of interviews with AI experts, humanitarian practitioners, and policy developers, the study identifies three key interconnected themes: AI risks in humanitarian work (data privacy, bias, ethical dilemmas), organizational readiness (literacy gaps, leadership, governance), and collaboration (cross-sector partnerships, ethical considerations). The authors propose an HCI-informed framework called ECHO (Educate, Co-create, Handhold, Optimize) to support responsible AI adoption by building ethical capacity, enhancing organizational readiness, and fostering equitable collaborations. The paper argues that HCI can guide the development of AI systems tailored to the complex demands of humanitarian contexts, ensuring ethical governance, capacity building, and participatory design for sustainable and accountable AI implementation.


**You:** summarize the paper

**Gemini:** This paper explores the potential and challenges of adopting Artificial Intelligence (AI) in humanitarian practice, emphasizing the crucial role of Human-Computer Interaction (HCI) principles. Through thematic analysis of interviews with AI experts, humanitarian practitioners, and policy developers, the study identifies three key interconnected themes: AI risks in humanitarian work (data privacy, bias, ethical dilemmas), organizational readiness (literacy gaps, leadership, governance), and collaboration (cross-sector partnerships, ethical considerations). The authors propose an HCI-informed framework called ECHO (Educate, Co-create, Handhold, Optimize) to support responsible AI adoption by building ethical capacity, enhancing organizational readiness, and fostering equitable collaborations. The paper argues that HCI can guide the development of AI systems tailored to the complex demands of humanitarian contexts, ensuring ethical governance, capacity building, and participatory design for sustainable and accountable AI implementation.


**You:** what are the approaches used to collect the data?

**Gemini:** The paper states that the data was collected through qualitative semi-structured interviews.
